In [320]:
import numpy as np
import pandas as pd
import random
import re

path = "AmongUs_Dataset/"

In [321]:
files = []
users_i = [0 for i in range(29)]
for i in range(29):
    tempdf = pd.read_csv('{}/User{}.csv'.format(path, i+1), delimiter = ',')
    tempdf['User_Id'] = i
    files.append(tempdf)
    users_i[i] = len(tempdf)
Dataset = pd.concat(files, ignore_index = True)

print(users_i)

[100, 100, 91, 100, 35, 100, 100, 100, 100, 100, 31, 100, 100, 100, 39, 70, 72, 100, 38, 64, 79, 100, 37, 38, 100, 33, 100, 54, 46]


In [322]:
Dataset

,Game Completed Date,Team,Outcome,Task Completed,All Tasks Completed,Murdered,Imposter Kills,Game Length,Ejected,Sabotages Fixed,Time to complete all tasks,Rank Change,Region/Game Code,User_Id
0,12/13/2020 at 1:26:56 am EST,Crewmate,Win,3,No,Yes,-,07m 04s,No,2.0,-,++,NA / WYMSBF,0
1,12/13/2020 at 1:17:42 am EST,Crewmate,Loss,7,Yes,No,-,16m 21s,No,1.0,09m 48s,--,NA / WYMSBF,0
2,12/13/2020 at 12:57:47 am EST,Crewmate,Win,3,No,No,-,11m 33s,No,0.0,-,++,NA / WYMSBF,0
3,12/13/2020 at 12:41:55 am EST,Imposter,Win,-,-,-,2,08m 05s,No,NaN,-,+++,Europe / QIRTNF,0
4,12/13/2020 at 12:30:37 am EST,Crewmate,Loss,4,No,No,-,05m 10s,No,0.0,-,---,Europe / QIRTNF,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,12/01/2020 at 11:07:41 am EST,Imposter,Loss,-,-,-,2,14m 10s,No,NaN,-,-,NA / SNNGZF,28
2223,12/01/2020 at 10:52:25 am EST,Crewmate,Win,7,Yes,No,-,14m 11s,No,1.0,14m 10s,++,NA / SNNGZF,28
2224,11/26/2020 at 11:23:14 am EST,Crewmate,Win,7,Yes,No,-,19m 45s,No,1.0,15m 16s,++,Europe / NZWLXQ,28
2225,11/26/2020 at 11:00:36 am EST,Crewmate,Win,7,Yes,No,-,10m 18s,No,0.0,06m 13s,++,Europe / NZWLXQ,28


In [323]:
def user_train_valid_split(users_i,valid_ratio,possible=False):
    n = sum(users_i)
    target = int(np.floor(valid_ratio*n))
    while not possible:
        possible, validation_indices = user_split(users_i,target)
        target -= 1
    training_indices = [i for i in range(len(users_i)) if i not in validation_indices]
    return training_indices, validation_indices

def user_split(users_i,target):  
    # Reference: https://levelup.gitconnected.com/dynamic-programming-subset-sum-c386126621cd
    n = len(users_i)
    solution = [[False for j in range(int(target+1))] for i in range(n+1)]
    # base cases
    for i in range(n):
        solution[i][0] = True
    # other cases
    for i in range(1,n+1):
        for j in range(1,target+1):
            solution[i][j] = solution[i-1][j]
            if(solution[i][j] == False and j >= users_i[i-1]):
                solution[i][j] = solution[i][j] or solution[i-1][j-users_i[i-1]]
    # check if the subset sum is possible
    possible = solution[len(users_i)][target]
    subset = []
    if not possible: return possible, subset
    # return the subset solution if one exists
    y = len(users_i)
    x = target
    while x != 0:
        if solution[y-1][x] == False:
            subset.append(y-1)
            x -= users_i[y-1]
        else: 
            y -= 1
    return possible, subset

In [324]:
valid_ratio = .25
training_indices, validation_indices = user_train_valid_split(users_i,valid_ratio)

#check
train_sum = sum([users_i[i] for i in training_indices])
valid_sum = sum([users_i[i] for i in validation_indices])
print("expected valid ratio: {}, actual: {}".format(valid_ratio, valid_sum/(train_sum+valid_sum)))

train_data = Dataset.loc[Dataset['User_Id'].isin(training_indices)]
valid_data = Dataset.loc[Dataset['User_Id'].isin(validation_indices)]
train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

expected valid ratio: 0.25, actual: 0.24966322406825325
